<a href="https://colab.research.google.com/github/2813/dls-homework/blob/main/%5Bhomework%5DAttention_and_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Глубокое обучение. Часть 2
# Домашнее задание по теме "Механизм внимания"

Это домашнее задание проходит в формате peer-review. Это означает, что его будут проверять ваши однокурсники. Поэтому пишите разборчивый код, добавляйте комментарии и пишите выводы после проделанной работы.

В этом задании вы будете решать задачу классификации математических задач по темам (многоклассовая классификация) с помощью Transformer.

В качестве датасета возьмем датасет математических задач по разным темам. Нам необходим следующий файл:

[Файл с классами](https://docs.google.com/spreadsheets/d/1IMRxByfg7gjoZ5i7rxvuNDvSrbdOJOc-/edit?usp=drive_link&ouid=104379615679964018037&rtpof=true&sd=true)

**Hint:** не перезаписывайте модели, которые вы получите на каждом из этапов этого дз. Они ещё понадобятся.

### Задание 1 (2 балла)

Напишите кастомный класс для модели трансформера для задачи классификации, использующей в качествке backbone какую-то из моделей huggingface.

Т.е. конструктор класса должен принимать на вход название модели и подгружать её из huggingface, а затем использовать в качестве backbone (достаточно возможности использовать в качестве backbone те модели, которые упомянуты в последующих пунктах)

In [ ]:
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

from datasets import Dataset
from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification, Trainer, TrainerArguments, AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers.modeling_outputs import TokenClassifierOutput

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
### This is just an interface example. You may change it if you want.

class TransformerClassificationModel(nn.Module):
    def __init__(base_transformer_model: Union[str, nn.Module], num_classes: int=7):
        self.backbone = AutoModelForSequenceClassification.from_pretrained(base_transformer_model)


        # YOUR CODE: create additional layers for classfication
        self.classifier = nn.Linear(backbone.config.hidden_size, num_classes)

    def forward(inputs, ...):
        # YOUR CODE: propagate inputs through the model. Return dict with logits
        outputs = self.backbone(input_ids, attention_mask)['last_hidden_state'][0][0,:]
        logits = self.classifier(outputs)

        return TokenClassifierOutput(
            loss=nn.CrossEntropyLoss(),
            logits=self.classifier(outputs)
            hidden_states=outputs,
            attentions=outputs.attentions
            )

### Задание 2 (1 балл)

Напишите функцию заморозки backbone у модели (если необходимо, возвращайте из функции модель)

In [ ]:
def freeze_backbone_function(model: TransformerClassificationModel):
    for p in model.backbone.parameters():
      p.requires_grad = False

    for p in model.classifier.parameters():
      p.requires_grad = True

### Задание 3 (2 балла)

Напишите функцию, которая будет использована для тренировки (дообучения) трансформера (TransformerClassificationModel). Функция должна поддерживать обучение с замороженным и размороженным backbone.

In [ ]:
import copy

def train_transformer(transformer_model, dataloader, freeze_backbone=True)
    model = copy.deepcopy(transformer_model)
    ### YOUR CODE IS HERE
    if freeze_backbone:
      freeze_backbone_funtions(model)

    tokenizer = AutoTokenizer.from_pretrained(model)
    data_collator = transformers.DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
    training_args = transformers.Seq2SeqTrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=2,
    )
    trainer = transformers.Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_loader,
        eval_dataset=test_loader,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    trainer.train()

    return model

### Задание 4 (1 балл)

Проверьте вашу функцию из предыдущего пункта, дообучив двумя способами
*cointegrated/rubert-tiny2* из huggingface.

In [ ]:
rubert_tiny_transformer_model = TransformerClassificationModel(AutoModel.from_pretrained('cointegrated/rubert-tiny2'), num_classes=7)
rubert_tiny_finetuned_with_freezed_backbone = train_transformer(rubert_tiny_transformer_model, train_loader freeze_backbone=True)

rubert_tiny_transformer_model = TransformerClassificationModel(AutoModel.from_pretrained('cointegrated/rubert-tiny2'), num_classes=7)
rubert_tiny_full_finetuned = train_transformer(rubert_tiny_transformer_model, train_loader, freeze_backbone=False)

### Задание 5 (1 балл)

Обучите *tbs17/MathBert* (с замороженным backbone и без заморозки), проанализируйте результаты. Сравните скоры с первым заданием. Получилось лучше или нет? Почему?

In [ ]:
### YOUR CODE IS HERE (probably, similar on the previous step)
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('tbs17/MathBert')
model = AutoModelForSequenceClassification.from_pretrained('tbs17/MathBert')

mathBert_model = TransformerClassificationModel(model, num_classes=7)
mathbert_frozen = train_transformer(mathBert_model, train_loader, freeze_backbone=True)

mathbert_unfrozen = train_transformer(mathBert_model, train_loader, freeze_backbone=False)

### Задание 6 (1 балл)

Напишите функцию для отрисовки карт внимания первого слоя для моделей из задания

In [ ]:
def draw_first_layer_attention_maps(attention_head_ids: List, text: str, model: TransformerClassificationModel):
    pass

### Задание 7 (1 балл)

Проведите инференс для всех моделей **ДО ДООБУЧЕНИЯ** на 2-3 текстах из датасета. Посмотрите на головы Attention первого слоя в каждой модели на выбранных текстах (отрисуйте их отдельно).

Попробуйте их проинтерпретировать. Какие связи улавливают карты внимания? (если в модели много голов Attention, то проинтерпретируйте наиболее интересные)

In [ ]:
### YOUR CODE IS HERE

### Задание 8 (1 балл)

Сделайте то же самое для дообученных моделей. Изменились ли карты внимания и связи, которые они улавливают? Почему?

In [ ]:
### YOUR CODE IS HERE